# EHT Tutorial 1: Simulating Measurements and Imaging for a Synthetic Image
In this tutorial, we show how to simulate observations from self-defined observing parameters (using `im.observe`) or from a previous observation (using `im.observe_same`, which we demonstrate with a real EHT M87 observation). We then show how to perform imaging using the `eht-imaging` algorithm.

## Environment setup

In [ ]:
# Install `eht-imaging` library.
!pip install ehtim

In [ ]:
# Download NumPY array of synthetic image.
!wget https://github.com/berthyf96/eht_tutorial/raw/refs/heads/main/rowan_m87.npy

# Download an EHT array from the `eht-imaging` repo.
!wget https://raw.githubusercontent.com/achael/eht-imaging/refs/heads/main/arrays/EHT2017.txt

# Download real M87 observation data.
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_095_lo_hops_netcal_StokesI.uvfits

In [ ]:
# Import libraries.
import ehtim as eh
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Simulating measurements

### Image object
An Image object contains the image array values and metadata (location, day, pixel size, etc.).

In [ ]:
# Load array of image pixel intensities.
imarr = np.load('rowan_m87.npy')

In [ ]:
# Create Image object.
im = eh.image.Image(
    image=imarr,   # image array
    psize=FILLIN,  # assume FOV = 367 uas; convert to radians with eh.RADPERUAS
    ra=12.51372,   # ra of M87
    dec=12.51372,  # dec of M87
    source='M87',  # source name
    mjd=57853      # day
)

In [ ]:
# Regrid image to have a smaller FOV and fewer pixels.
im = im.regrid_image(
    targetfov=128 * eh.RADPERUAS,
    npix=256)

# Display image.
im.display();

In [ ]:
# Save image as a FITS file.
# Images can also be saved/loaded as text and HDF5 files.
im.save_fits('rowan_m87.fits')

# Load back image from the FITS file.
im = eh.image.load_image('rowan_m87.fits')

### Array object
An Array object contains metadata about the telescope array, including site locations, SEFDs, etc.
* **site**: name of the telescope site
* **x, y, z**: telescope's fixed location
* **sefdr, sefdl**: system equvalent flux density (SEFD) for the right and left circular polarization, respectively (lower SEFD means higher sensitivity)
* **dr, dl**: polarization leakage (i.e., how much signal from one polarization leaks into the other) for right and left circular polarization, respectively
* **fr_par, fr_elev, fr_off**: used to compute the feed angle (i.e., how the physical receiver that detects incoming radiation is rotated relative to the sky's polarization)

In [ ]:
# Load EHT array used in 2017.
array = eh.array.load_txt('EHT2017.txt')

# Look at information about the telescopes in the array.
array_df = pd.DataFrame(array.tarr)
array_df

### Simulate with `im.observe`

In [ ]:
# Generate an observation with self-defined observing parameters.
obs = im.observe(
  array=array,        # observing telescope array
  tint=60,            # length of time in seconds that data is integrated to
                      # get a measurement
  tadv=600,           # length of time in seconds between measurements
  tstart=0,           # start time of the observation in hours (UTC)
  tstop=24,           # stop time of the observation in hours (UTC)
  bw=4e9,             # observing bandwidth in Hz
  ttype='fast',       # Fourier transform type
                      # ('nfft' would be faster but we don't have pyNFFT)
  add_th_noise=True,  # whether to add thermal (Gaussian) noise on visibilities
  phasecal=True,      # whether to calibrate station-dependent phase
  ampcal=True         # whether to calibrate station-dependent gain
)

### Look at plots

In [ ]:
# Plot uv coverage.
obs.plotall('u', 'v', conj=True);

In [ ]:
# Plot amplitude with baseline u-v distance.
# Note the 1/f dropoff in amplitude vs. u-v distance (which directly
# corresponds to frequency).
obs.plotall('uvdist', 'amp');

In [ ]:
# Plot phase with baseline distance.
obs.plotall('uvdist', 'phase');

### Simulate an observation based on real M87 data

In [ ]:
# Load M87 observation.
obs_M87_orig = eh.obsdata.load_uvfits(
    'SR1_M87_2017_095_lo_hops_netcal_StokesI.uvfits')

# We have to change the image coordinates (ra, dec) and frequency (rf)
# to match the observed coordinates and frequency.
im_M87 = im.copy()
im_M87.ra = obs_M87_orig.ra
im_M87.dec = obs_M87_orig.dec
im_M87.rf = obs_M87_orig.rf

In [ ]:
# Simulate an observation with the same settings as those of the
# M87 observation (assuming calibrated amplitudes and phases).
obs_M87_calib = im_M87.observe_same(
    obs_M87_orig,
    add_th_noise=True,
    phasecal=True,
    ampcal=True,
    ttype='fast')

In [ ]:
# Look at uv coverage of M87 observation.
obs_M87_calib.plotall('u', 'v', conj=True);

### Simulate more realistic M87-based observation

In [ ]:
# Add scan info to Obsdata object so that per-scan stabilization and averaging
# can be done.
obs_M87_orig.add_scans()

# Std. dev. of the constant absolute gain of each telescope from a gain of 1:
gain_offset = {'AA': 0.1, 'AP': 0.1, 'AZ': 0.1, 'LM': 0.6, 'PV': 0.1, 'SM': 0.1,
               'JC': 0.1, 'SP': 0.1, 'SR': 0.0}
# Std. dev. of the time-varying gain differences:
gainp = {'AA': 0.05, 'AP': 0.05, 'AZ': 0.05, 'LM': 0.5, 'PV': 0.05, 'SM': 0.05,
         'JC': 0.05, 'SP': 0.15, 'SR': 0.0}

In [ ]:
# Simulate an observation with phase and amplitude (gain) errors.
obs_M87 = im_M87.observe_same(
    obs_M87_orig,
    add_th_noise=True,
    phasecal=False,
    ampcal=False,
    stabilize_scan_phase=True,  # `True` makes phase error constant per scan
    stabilize_scan_amp=True,    # `True` makes gain error constant per scan
    gain_offset=gain_offset,    # use station-dependent median gain error
    gainp=gainp,                # use station-dependent variability
    ttype='fast')

In [ ]:
# Compare phases with vs. without atmospheric noise.
eh.plotting.comp_plots.plotall_obs_compare(
  [obs_M87_calib, obs_M87], 'uvdist', 'phase', ebar=False,
  legendlabels=['calibrated', 'realistic']);

In [ ]:
# Compare closure phases on a triangle with vs. without atmospheric noise.
eh.plotting.comp_plots.plot_cphase_obs_compare(
  [obs_M87_calib, obs_M87], 'AA', 'PV', 'LM', ebar=False,
  legendlabels=['calibrated', 'realistic']);

## Imaging

### Intrinsic resolution
The intrinsic, or nominal, resolution of the array is proptional to $\lambda/\lvert\mathbf{u}\rvert_{\max}$. It determines the finest angular resolution that can be recovered from the data. Any image features above this resolution come from super-resolution.

In [ ]:
res = obs_M87.res()  # nominal resolution ∝ 1 / longest baseline
print(f'Nominal resolution: {res / eh.RADPERUAS:.1f} uas')

In [ ]:
# Show the true image blurred to the nominal resolution.
blurry_im = im_M87.blur_circ(res)
blurry_im.display();

### Dirty image
The dirty image is the inverse Fourier transform of the zero-filled raw visibilities.
When there's phase information, it's equal to the sky image convolved with the dirty beam.

In [ ]:
# Show the dirty beam (a.k.a. PSF) of the observation.
dbeam = obs_M87.dirtybeam(npix=64, fov=128 * eh.RADPERUAS)
dbeam.display();

In [ ]:
# Show the dirty image (inverse Fourier transform of the raw visibilities).
dim = obs_M87_calib.dirtyimage(npix=64, fov=128 * eh.RADPERUAS)
dim.display();

### Scan-average observation data

In [ ]:
# Make sure Obsdata objects have scan info.
obs_M87_calib.add_scans()

# Reduce each scan down to one average measurement.
obs_M87_calib_scan_avg = obs_M87_calib.avg_coherent(0, scan_avg=True)
obs_M87_scan_avg = obs_M87.avg_coherent(0, scan_avg=True)

# Compare data before vs. after scan-averaging.
eh.plotting.comp_plots.plotall_obs_compare(
  [obs_M87_calib, obs_M87_calib_scan_avg], 'uvdist', 'amp',
  legendlabels=['original', 'scan-averaged']);

### Imaging parameters

In [ ]:
# Image parameters:
npix = 64
fov = 128 * eh.RADPERUAS
# Here we take the true total flux density, but when it is not known,
# it can be estimated from the zero baseline, e.g. with
# `zbl = np.median(obs.unpack_bl('ALMA', 'APEX', 'amp')['amp']).item()`.
zbl = im.total_flux()
print(zbl)

In [ ]:
# Use a Gaussian blob for the image initialization.
gauss_fwhm = 80 * eh.RADPERUAS
emptyim = eh.image.make_square(obs_M87, npix, fov)
gaussim = emptyim.add_gauss(zbl, (gauss_fwhm, gauss_fwhm, 0, 0, 0))

# Make the initial image have the correct coordinates and reference frequency.
gaussim.source = 'M87'
gaussim.ra = obs_M87.ra
gaussim.dec = obs_M87.dec
gaussim.rf = obs_M87.rf

gaussim.display();

### Imaging from the calibrated observation

In [ ]:
# We can use just the complex visibilities because they are well-calibrated.
data_term = {'vis': 1.}
# We don't need strong regularization because the complex visibilities are
# already well constraining.
reg_term = {'tv2': 0.1}

# Set up imager.
imgr = eh.imager.Imager(
  obs_M87_calib_scan_avg,
  init_im=gaussim,
  flux=zbl,
  data_term=data_term,
  reg_term=reg_term,
  norm_reg=True,
  maxit=300,  # max. number of L-BFGS-B iterations to minimize RML objective
  ttype='fast')

In [ ]:
# Run multiple rounds of imaging. Each round involves
# 1. Minimize RML objective using L-BFGS-B optimizer.
# 2. Blur the result to the intrinsic resolution
#    and use it as initialization for the next stage.

# Imaging round 1:
imgr.make_image_I(show_updates=False)
out1 = imgr.out_last()

# Imaging round 2:
imgr.init_next = imgr.out_last().blur_circ(res)
imgr.make_image_I(show_updates=False)
out2 = imgr.out_last()

In [ ]:
# Show the results of the imaging rounds.
outs = [out1, out2]
fig, axs = plt.subplots(1, len(outs), figsize=(5 * len(outs), 3.5))
for i, out in enumerate(outs):
  ax = axs[i]
  mappable = ax.imshow(out.ivec.reshape(npix, npix), cmap='afmhot')
  fig.colorbar(mappable, ax=ax)
  ax.axis('off')
  ax.set_title(f'Round {i + 1}')
plt.show()

In [ ]:
# Look at fit to phases.
eh.plotting.comp_plots.plotall_obs_im_compare(
    obs_M87_calib_scan_avg, out, 'uvdist', 'phase', ttype='fast');

In [ ]:
# Look at fit to amplitudes.
eh.plotting.comp_plots.plotall_obs_im_compare(
    obs_M87_calib_scan_avg, out, 'uvdist', 'amp', ttype='fast');

In [ ]:
# Define a flat image prior to potentially use as an initial image.
flatim = eh.image.make_square(obs_M87, npix, fov)
flatim.imvec = np.ones(npix * npix) * (zbl / (npix**2))

### Imaging from realistic M87-based observation

In [ ]:
# Inflate amplitude error bars with systematic noise, which accounts for
# gain errors, polarization leakage, etc.

# The following systematic noise estimates are taken from the fiducial
# eht-imaging pipeline.
systematic_noise = {
    'AA': 0.01220,
    'AP': 0.01339,
    'AZ': 0.00860,
    'LM': 0.17613,  # LMT had the most variability
    'PV': 0.01220,
    'SM': 0.01810,
    'JC': 0.01693,
    'SP': 0.00860
}

In [ ]:
# The following values are taken from the fiducial eht-imaging pipeline.
data_term = {'amp': 0.2, 'cphase': 1., 'logcamp': 1.}
reg_term = {
    'simple': 100,  # maximum entropy (similarity to Gaussian prior)
    'tv': 1,        # total variation
    'tv2': 1,       # total squared variation
    'flux': 1e4     # compact flux constraint
}

# To avoid gradient singularities associated in the first step:
gaussim = gaussim.add_gauss(
    zbl * 1e-3, (gauss_fwhm, gauss_fwhm, 0, gauss_fwhm, gauss_fwhm))

# Set up imager.
imgr = eh.imager.Imager(
  obs_M87_scan_avg,
  init_im=gaussim,
  prior_im=gaussim,
  flux=zbl,
  data_term=data_term,
  reg_term=reg_term,
  norm_reg=True,
  systematic_noise=systematic_noise,
  maxit=100,
  ttype='fast')

In [ ]:
# Just run one round of imaging for demonstration purposes
# (it already gets us most of the way there).
imgr.make_image_I(show_updates=False)

# Show the result of imaging.
out = imgr.out_last()
out.display();

In [ ]:
# Try imaging with a smaller Gaussian blob for the initialization.
gauss_fwhm_small = 40 * eh.RADPERUAS
emptyim = eh.image.make_square(obs_M87_orig, npix, fov)
gaussim_small = emptyim.add_gauss(
    zbl, (gauss_fwhm_small, gauss_fwhm_small, 0, 0, 0))
gaussim_small = gaussim_small.add_gauss(
    zbl * 1e-3, (gauss_fwhm_small, gauss_fwhm_small, 0,
                 gauss_fwhm_small, gauss_fwhm_small))
gaussim_small.display();

In [ ]:
# Set up imager.
imgr = eh.imager.Imager(
  obs_M87_scan_avg,
  init_im=gaussim_small,
  prior_im=gaussim_small,
  flux=zbl,
  data_term=data_term,
  reg_term=reg_term,
  norm_reg=True,
  systematic_noise=systematic_noise,
  maxit=100,
  ttype='fast')

# Run one round of imaging.
imgr.make_image_I(show_updates=False)

# Show the result of imaging.
out = imgr.out_last()
out.display();